Initialize parser

In [4]:
from tree_sitter import Language, Parser
from pathlib import Path

LIB = "../../build/tree-sitter-binaries/diff.so"
DIFF_LANGUAGE = Language(LIB, "diff")

parser = Parser()
parser.set_language(DIFF_LANGUAGE)

query = DIFF_LANGUAGE.query("""
(deletion) @deletion
(addition) @addition
""")

deletions = []
additions = []

Parsing Diff #1 - Material

In [5]:
diff_1 = Path("../../resources/golden-reference/claude-sonnet-4_5/test4-deprecated-material.diff").read_text()

tree = parser.parse(bytes(diff_1, 'utf-8'))
captures = query.captures(tree.root_node)

for node, capture_name in captures:
    text = node.text.decode('utf-8')
    # remove - or + character and leading & trailing whitespaces
    content = text[1:].strip() if len(text) > 1 else ""

    if capture_name == 'deletion':
        deletions.append(content)
    elif capture_name == 'addition':
        additions.append(content)

print("Deletions:")
for d in deletions:
    print(d)

print("\nAdditions:")
for a in additions:
    print(a)

print(f"has error: {tree.root_node.has_error}")
print(f"s-exp: {tree.root_node.sexp()}")

required_deletions = [
    "colors = TopAppBarDefaults.centerAlignedTopAppBarColors(",
    "colors: TopAppBarColors = TopAppBarDefaults.centerAlignedTopAppBarColors(),",
]

required_additions = [
    "colors = TopAppBarDefaults.topAppBarColors(",
    "colors: TopAppBarColors = TopAppBarDefaults.topAppBarColors(),",
]

contains_all_deletions = all(
    any(req in deletion for deletion in deletions)
    for req in required_deletions
)

contains_all_additions = all(
    any(req in addition for addition in additions)
    for req in required_additions
)

print("contains_all_deletions:", contains_all_deletions)
print("contains_all_additions:", contains_all_additions)

deletions.clear()
additions.clear()

Deletions:
colors: TopAppBarColors = TopAppBarDefaults.centerAlignedTopAppBarColors(),
colors = TopAppBarDefaults.centerAlignedTopAppBarColors(

Additions:
colors: TopAppBarColors = TopAppBarDefaults.topAppBarColors(),
colors = TopAppBarDefaults.topAppBarColors(
has error: False
s-exp: (source (old_file (filename)) (new_file (filename)) (location (linerange) (linerange)) (context) (context) (context) (deletion) (addition) (context) (context) (context) (old_file (filename)) (new_file (filename)) (location (linerange) (linerange)) (context) (context) (context) (deletion) (addition) (context) (context) (context))
contains_all_deletions: True
contains_all_additions: True


Parsing Diff #2 - Plugin

In [6]:
diff_2 = Path("../../resources/golden-reference/claude-sonnet-4_5/test5-deprecated-plugin.diff").read_text()

tree = parser.parse(bytes(diff_2, 'utf-8'))
captures = query.captures(tree.root_node)

for node, capture_name in captures:
    text = node.text.decode('utf-8')
    # remove - or + character and leading & trailing whitespaces
    content = text[1:].strip() if len(text) > 1 else ""

    if capture_name == 'deletion':
        deletions.append(content)
    elif capture_name == 'addition':
        additions.append(content)

print("Deletions:")
for d in deletions:
    print(d)

print("\nAdditions:")
for a in additions:
    print(a)


required_deletions = [
    "import org.jetbrains.kotlin.gradle.dsl.KotlinTopLevelExtension",
    "private inline fun <reified T : KotlinTopLevelExtension> Project.configureKotlin() = configure<T> {",
]

required_additions = [
    "import org.jetbrains.kotlin.gradle.dsl.KotlinBaseExtension",
    "private inline fun <reified T : KotlinBaseExtension> Project.configureKotlin() = configure<T> {"
]

contains_all_deletions = all(
    any(req in deletion for deletion in deletions)
    for req in required_deletions
)

contains_all_additions = all(
    any(req in addition for addition in additions)
    for req in required_additions
)

print(f"has error: {tree.root_node.has_error}")
print(f"s-exp: {tree.root_node.sexp()}")

print("contains_all_deletions:", contains_all_deletions)
print("contains_all_additions:", contains_all_additions)

Deletions:
import org.jetbrains.kotlin.gradle.dsl.KotlinJvmProjectExtension
import org.jetbrains.kotlin.gradle.dsl.KotlinTopLevelExtension
private inline fun <reified T : KotlinTopLevelExtension> Project.configureKotlin() = configure<T> {

Additions:
import org.jetbrains.kotlin.gradle.dsl.KotlinBaseExtension
import org.jetbrains.kotlin.gradle.dsl.KotlinJvmProjectExtension
private inline fun <reified T : KotlinBaseExtension> Project.configureKotlin() = configure<T> {
has error: False
s-exp: (source (old_file (filename)) (new_file (filename)) (location (linerange) (linerange)) (context) (context) (context) (deletion) (deletion) (addition) (addition) (context) (location (linerange) (linerange)) (context) (context) (context) (deletion) (addition) (context) (context) (context))
contains_all_deletions: True
contains_all_additions: True
